Reference:</br>
https://github.com/dansuh17/alexnet-pytorch/blob/d0c1b1c52296ffcbecfbf5b17e1d1685b4ca6744/model.py#L40</br>
https://github.com/pytorch/vision/blob/main/torchvision/models/alexnet.py

In [20]:
import os
import tarfile
import requests
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils import data
from torchsummary import summary

In [2]:
# Current working directory.
!pwd

/root/jupyter_projects/PyTorch-Zoo/AlexNet


In [7]:
# Change this for your own directory.
ROOT_DIR = Path("/root/jupyter_projects")  # Your working directory.
DATASET_DIR = ROOT_DIR / "Data"  # Folder to store dataset.

data_path = DATASET_DIR / "Flowers102"  # Path of your dataset.

In [8]:
print(ROOT_DIR)
print(DATASET_DIR)
print(data_path)

/root/jupyter_projects
/root/jupyter_projects/Data/Flowers102-builtin


In [16]:
url = "https://www.dropbox.com/s/hqiryv0g62lp878/oxford-102-flowers.tgz?raw=1"
file_name = "oxford-102-flowers.tgz"

r = requests.get(url, allow_redirects=True)
open(file_name, 'wb').write(r.content)
print('Dataset created')

In [ ]:
path_to_zip_file = data_path / 
directory_to_extract_to = DATASET_DIR

with tarfile.open(path_to_zip_file, "w") as tar:
    tar.extractall(directory_to_extract_to)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Define pytorch device.

In [17]:
NUM_EPOCHS = 10
BATCH_SIZE = 64
IMAGE_DIM = 227  # Image dimension: (3, 227, 227)
NUM_CLASSES = 102

data_path = DATASET_DIR

In [18]:
print(data_path)

/root/jupyter_projects/Data/Flowers102-builtin


In [19]:
print(len(next(os.walk(data_path))[1]))

1


In [12]:
img_transforms = transforms.Compose([
    transforms.CenterCrop(IMAGE_DIM),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]),
])

Image for each class is stored in a separate folder, we can use `datasets.ImageFolder` to create dataset.

In [13]:
dataset = datasets.ImageFolder(
    root=data_path, 
    transform=img_transforms
)
print(f"Total number of samples: {len(dataset)}")

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
# Split the data into training set and validation set.
train_data_size = int(len(dataset) * 0.8)
val_data_size = len(dataset) - train_data_size
train_data, val_data = data.random_split(dataset, [train_data_size, val_data_size])

In [ ]:
print(f"Total number of samples in training set: {len(train_data)}")
print(f"Total number of samples in validation set: {len(val_data)}")

In [ ]:
train_dataloader = data.DataLoader(train_data, batch_size=BATCH_SIZE)
val_dataloader = data.DataLoader(val_data, batch_size=BATCH_SIZE)

In [ ]:
print(f"Number of iterations in training set: {len(train_dataloader)}")
print(f"Number of iterations in validation set: {len(val_dataloader)}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for batch, (X, y) in enumerate(train_dataloader):
    print(batch)
    print(X.shape)
    
    plt.imshow(X[17].permute(1, 2, 0))
    plt.show()
    
    print(y.shape)
    print(y[17])
    break

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = NUM_CLASSES, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),  # !
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
alexnet = AlexNet()

In [ ]:
summary(alexnet, (3, 227, 227))

In [ ]:
seed = torch.initial_seed()
print('*Used seed : {}*'.format(seed))  # seed value

In [ ]:
optimizer = optim.Adam(params=alexnet.parameters(), lr=0.0001)
print('Optimizer created')

In [ ]:
loss_fn = nn.CrossEntropyLoss()
print('Lost function created')

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Total number of samples in training set.
    model.train()
    for batch, (X, y) in enumerate(dataloader):  # Iterations.
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error.
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        
        # Backpropagation.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)  # Iteration * Batch Size = Number of samples have being trained.
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)  # Total number of samples in test set.
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
model = alexnet

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")